In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings

warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random

pd.set_option('display.max_columns', None)

In [2]:
# 저장할 공간
path_master = "항적데이터"

In [3]:
path_list = os.listdir(path_master)
path_list

['AKADIMOS_voyage_df.csv',
 'AKA_BHUM_voyage_df.csv',
 'AMALTHEA_voyage_df.csv',
 'CAPE_ARTEMISIO_voyage_df.csv',
 'CAUQUENES_voyage_df.csv',
 'CAUTIN_voyage_df.csv',
 'CISNES_voyage_df.csv',
 'COCHRANE_voyage_df.csv',
 'COPIAPO_voyage_df.csv',
 'COSCO_ASIA_voyage_df.csv',
 'COSCO_FOS_voyage_df.csv',
 'COSCO_PRINCE_RUPERT_voyage_df.csv',
 'COYHAIQUE_voyage_df.csv',
 'CSCL_ASIA_voyage_df.csv',
 'CSCL_AUTUMN_voyage_df.csv',
 'CSCL_WINTER_voyage_df.csv',
 'filtered_ais.csv',
 'filtered_ais_2.csv',
 'HENG_HUI_6_voyage_df.csv',
 'HEUNG-A_ULSAN_voyage_df.csv',
 'HMM_BLESSING_voyage_df.csv',
 'HMM_RAON_voyage_df.csv',
 'HYUNDAI_SATURN_voyage_df.csv',
 'ITAJAI_EXPRESS_voyage_df.csv',
 'KUALA_LUMPUR_EXPRESS_voyage_df.csv',
 'MANZANILLO_EXPRESS_voyage_df.csv',
 'MERSIN_EXPRESS_voyage_df.csv',
 'MSC_ABY_voyage_df.csv',
 'MSC_AGRIGENTO_voyage_df.csv',
 'MSC_ALGHERO_voyage_df.csv',
 'MSC_ALICE_voyage_df.csv',
 'MSC_ALINA_voyage_df.csv',
 'MSC_ALIYA_voyage_df.csv',
 'MSC_AMEERA_III_voyage_df.csv',
 

In [4]:
filtered_df=pd.read_csv("항적데이터/filtered_ais_2.csv")
filtered_df

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
0,RIO DE JANEIRO EXPRESS,211111000,9508902.0,0,KRBNP,9150500,-147.2260,43.2993,18.9,293.0,2023-01-14 10:15:42.000000,05:42.0,36:18.0,under way using engine,0.000000,11878.789690
1,UMM SALAL,215237000,9525857.0,13296,CNTAO,9071200,121.0697,35.7353,0.0,173.0,2023-01-14 10:15:42.000000,05:55.0,53:55.0,at anchor,0.000000,NaN
2,KUALA LUMPUR EXPRESS,218284000,9343730.0,8606,CLIQQ,9040500,-70.1564,-20.2053,0.0,209.0,2023-01-14 10:15:42.000000,09:09.0,05:09.0,moored,0.000000,NaN
3,MSC LA SPEZIA,218400000,9461403.0,14736,CAVAN,9100500,-126.0553,48.1417,0.7,248.0,2023-01-14 10:15:42.000000,08:02.0,07:39.0,under way using engine,0.000000,203343.739100
4,NAVIGARE COLLECTOR,219059000,9779771.0,10100,CLVAP,9060400,-72.7564,-34.2232,18.2,19.0,2023-01-14 10:15:42.000000,09:43.0,22:58.0,under way using engine,0.000000,63074.783850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544923,ITAJAI EXPRESS,538009762,9520053.0,13312,YANTIAN,1130900,114.6343,22.3504,6.3,330.0,2023-01-14 01:19:10.336955,2023-01-13 16:09:46.000000,2023-01-13 07:54:29.000000,under way using engine,1966.621201,253.966024
544924,MSC SKY II,636016574,9162277.0,2169,ZHUSHAN SHIPYARD,11041000,122.2400,29.9456,0.0,111.0,2023-01-14 01:19:10.336955,2023-01-13 16:06:50.000000,2023-01-13 04:16:32.000000,moored,832.167765,NaN
544925,MSC RINI III,636021011,9419644.0,2664,PANJANG,1140100,105.3790,-5.6416,14.7,324.0,2023-01-14 01:19:10.336955,2023-01-13 16:17:37.000000,2023-01-13 04:16:00.000000,under way using engine,5139.842319,321.893997
544926,MSC RINI III,636021011,9419644.0,2664,PANJANG,1140100,105.3753,-5.6364,14.7,324.0,2023-01-14 01:19:48.516171,2023-01-13 16:19:07.000000,2023-01-13 04:16:00.000000,under way using engine,5139.511258,321.873263


In [5]:
filtered_df["vessel_name"].value_counts()

ZHU CHENG XIN ZHOU    16901
COSCO FOS             15257
HEUNG-A ULSAN         13743
MSC GIANNINA II       13511
SINOKOR NIIGATA       13297
                      ...  
MSC SKY II               60
MSC CAPELLA              57
UMM SALAL                55
YM WELLBEING             50
MSC BEIJING              49
Name: vessel_name, Length: 197, dtype: int64

In [6]:
port_mis = pd.read_csv("port_mis_processed.csv")
port_mis

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,총톤수,입항일시,출항일시,CIQ수속일자,수리일시,항해구분,MRN 번호,계선장소,계선장소.1,계선장소.2,차항지,전출항지,선박용도,외항:한국인선원수\n내항:해기사선원수,외항:외국인선원수\n내항:보통선원수,승객,예선,도선,부선호출부호1,부선호출부호2
0,부산,VRAD2,OSG ALPHA,2015,8,최종,외항,출항,"7,167",2015-09-24 16:00,2022-08-10 00:00,2015-09-24 19:00,2015-10-06 11:27,기타,15ANJSM0229,WAM,08,M박지 M8 묘지,DALIAN,MAIZURU,풀컨테이너선,NaN,17.0,NaN,N,N,NaN,NaN
1,부산,140025,한림,2015,23,최종,내항,출항,432,2015-09-25 11:40,2022-07-31 00:00,NaN,2015-10-08 10:25,기타,NaN,WAN,01,남외항 N-1박지,온산,제주,석유제품 운반선,5.0,2.0,NaN,N,N,NaN,NaN
2,부산,030968,삼원골드,2017,93,변경,내항,출항,790,2017-11-01 04:50,2023-01-11 23:00,NaN,NaN,기타,NaN,MW5,01,5물량장 1선석,기타항,기타항,석유제품 운반선,4.0,4.0,0.0,N,N,NaN,NaN
3,부산,D9JC,성신티3호,2019,7,최종,내항,출항,106.21,2019-03-21 12:50,2022-11-01 13:35,NaN,2022-11-02 09:45,기타,NaN,*LP,01,대평동호안,기타항,기타항,기타 예선,4.0,0.0,0.0,N,N,NaN,NaN
4,부산,UCXC,CAESAR,2019,2,변경,외항,출항,685,2019-12-29 15:17,2022-12-31 18:00,2022-12-31 18:00,NaN,기타,21TNICD005E,MYQ,01,기타 북항사설조선소,OCEAN DISTRICT,속초,냉동.냉장선,NaN,2.0,NaN,Y,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42633,부산,220031,말리,2023,1,최초,내항,입항,"2,692",2023-01-13 21:00,2023-01-14 05:00,NaN,NaN,기타,NaN,MBE,Q1,감만부두 동측안벽 1 선석,울산,대산,석유제품 운반선,8.0,8.0,0.0,Y,N,NaN,NaN
42634,부산,9HA4999,CMA CGM J ADAMS,2023,1,최종,외항,입항,"141,950",2023-01-13 21:00,2023-01-14 19:00,2023-01-13 21:00,NaN,기타,23CMAL0004I,MS5,04,신항 5부두 4선석,SHEKOU PT,YOKOHAMA,풀컨테이너선,NaN,28.0,NaN,Y,Y,NaN,NaN
42635,부산,A8SI7,ZIM SAN DIEGO,2023,1,최종,외항,입항,"91,158",2023-01-13 21:00,2023-01-14 19:00,2023-01-13 21:00,NaN,기타,23ZIMU0020I,MSN,05,신항 2부두 2선석,"HOUSTON, TX",YANTIAN PT,풀컨테이너선,NaN,20.0,NaN,Y,Y,NaN,NaN
42636,부산,3FNR5,DONG YU,2023,1,변경,외항,입항,"2,962",2023-01-13 23:00,2023-01-14 12:00,2023-01-13 23:30,NaN,기타,23SHSBM011I,WAN,03,남외항 N-3박지,SAKAISENBOKU,RIZHAO PT,일반화물선,NaN,13.0,NaN,N,N,NaN,NaN


In [7]:
port_mis["선명"] = port_mis["선명"].apply(lambda x : x.strip())

In [8]:
port_mis_df = port_mis[[
        "항명", "호출부호", "선명", "입항횟수", "입항횟수.1", "구분", '외내', '입출', '입항일시', '출항일시',
        "계선장소", '계선장소.1', '계선장소.2', '차항지', '전출항지'
    ]]

In [9]:
port_mis_df["입항일시"] = pd.to_datetime(port_mis_df["입항일시"])
port_mis_df["출항일시"] = pd.to_datetime(port_mis_df["출항일시"])

In [11]:
for path in path_list:
    voyage_df = pd.read_csv(path_master + "/" + path)
    voyage_df["vessel_name"] = voyage_df["vessel_name"].apply(lambda x: x.strip())
    vessel_name = voyage_df["vessel_name"].values[0]
    if not len(port_mis_df[port_mis_df["선명"] == vessel_name]) % 2 == 0:
        print(vessel_name)

CAPE ARTEMISIO
MSC ALINA
MSC COLETTE III
MSC JASPER VIII


In [13]:
for path in path_list:
    if path == "filtered_ais.csv" or path == 'filtered_ais_2.csv':
        continue
        
    voyage_df = pd.read_csv(path_master + "/" + path)
    voyage_df["vessel_name"] = voyage_df["vessel_name"].apply(lambda x: x.strip())
    vessel_name = voyage_df["vessel_name"].values[0]
    
    voyage_df["timestamp"] = pd.to_datetime(voyage_df["timestamp"], errors='coerce')
#     voyage_df.dropna(subset=["timestamp"], inplace=True)
#     voyage_df.reset_index(drop=True, inplace=True)
    
    pmis_df = port_mis_df[port_mis_df.선명 == vessel_name]
    if len(pmis_df) == 0:
        print("port mis 데이터 없는 선박 : ",vessel_name)
        continue
    pmis_df.reset_index(inplace=True, drop=True)
    
    port_mis_voy_dict = {
        "voyage_name": [],
        "port_in_time": [],
        "port_out_time": [],
        "next_port": [],
        "previous_port": []
    }
    
    
    
    vessel_name = vessel_name.replace(" ", "_")
    
    # 입항횟수.1 = ser_no임. 
    # ser_no가 같은 것들
    for j in set(pmis_df["입항횟수"]):
        ff = pmis_df[pmis_df["입항횟수"] == j]
        for i in set(ff["입항횟수.1"]):
            d = ff[ff["입항횟수.1"] == i]
            voyage_name = d["호출부호"].values[0] +"_"+ str(d["입항횟수"].values[0]) +  "_" + str(d["입항횟수.1"].values[0])
            port_in_time = d['입항일시'].values[0]
            port_out_time = d['출항일시'].values[0]
            next_port = d['차항지'].values[0]
            previous_port = d['전출항지'].values[0]

            port_mis_voy_dict["voyage_name"].append(voyage_name)
            port_mis_voy_dict["port_in_time"].append(port_in_time)
            port_mis_voy_dict["port_out_time"].append(port_out_time)
            port_mis_voy_dict["next_port"].append(next_port)
            port_mis_voy_dict["previous_port"].append(previous_port)

        
                
    port_mis_voyage_df = pd.DataFrame(port_mis_voy_dict)
    port_mis_voyage_df = port_mis_voyage_df.sort_values(by='port_in_time', ignore_index=True)
    port_mis_voyage_df.to_csv("voy_path/" + vessel_name+ "_portmis" + ".csv",
                  encoding="utf-8 sig",
                  header=True,
                  index=False)
    
    for idx in port_mis_voyage_df.index:
        if idx == 0:
            voyage_name = port_mis_voyage_df.loc[idx, "voyage_name"]
            port_in_time = pd.to_datetime(port_mis_voyage_df.loc[idx, "port_in_time"])
            
            df = voyage_df[(voyage_df["timestamp"] < datetime.datetime.fromtimestamp(port_in_time.timestamp()))]
        else:
            port_in_time = pd.to_datetime(
                port_mis_voyage_df.loc[idx, "port_in_time"])
            prev_out_time = pd.to_datetime(
                port_mis_voyage_df.loc[idx - 1, "port_out_time"])
            #         print(port_in_time)
            voyage_name = port_mis_voyage_df.loc[idx, "voyage_name"]
            df = voyage_df[(voyage_df["timestamp"] < datetime.datetime.
                            fromtimestamp(port_in_time.timestamp()))
                           & (voyage_df["timestamp"] > datetime.datetime.
                              fromtimestamp(prev_out_time.timestamp()))]
        df.to_csv("voy_path/" + vessel_name + "_" + voyage_name +".csv", encoding="utf-8 sig", header=True, index=False)
    

port mis 데이터 없는 선박 :  ITAJAI EXPRESS
port mis 데이터 없는 선박 :  MSC ALICE
port mis 데이터 없는 선박 :  MSC FIE X
port mis 데이터 없는 선박 :  MSC INDIA
port mis 데이터 없는 선박 :  MSC NATASHA
port mis 데이터 없는 선박 :  MSC QINGDAO F
port mis 데이터 없는 선박 :  MSC SINDY
port mis 데이터 없는 선박 :  MSC SKY II


- 0번 인덱스
    - timestamp가 port_in_time 보다 작은 것들만 확인하여 0번 인덱스의 voyage_name
- n번 ~
    - n번 인덱스의 port_in_time ~ n-1번 인덱스의 port_out_time 사이에 있는 timestamp

In [178]:
d = pd.read_csv(path_master + "/COSCO_FOS_voyage_df.csv").sort_values(by="timestamp")
d["vessel_name"].values[0]

'COSCO FOS'

In [179]:
port_mis_df[port_mis_df.선명 == d["vessel_name"].values[0]]

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
1174,부산,VRKR6,COSCO FOS,2022,12,최종,외항,입항,2022-07-22 06:48:00,2022-07-23 17:56:00,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
1175,부산,VRKR6,COSCO FOS,2022,12,최종,외항,출항,2022-07-22 06:48:00,2022-07-23 17:56:00,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
3557,부산,VRKR6,COSCO FOS,2022,13,최종,외항,출항,2022-08-01 01:03:00,2022-08-02 17:56:00,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
3558,부산,VRKR6,COSCO FOS,2022,13,최종,외항,입항,2022-08-01 01:03:00,2022-08-02 17:56:00,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
5700,부산,VRKR6,COSCO FOS,2022,14,최종,외항,출항,2022-08-10 01:54:00,2022-08-11 14:54:00,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
5701,부산,VRKR6,COSCO FOS,2022,14,최종,외항,입항,2022-08-10 01:54:00,2022-08-11 14:54:00,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
7480,부산,VRKR6,COSCO FOS,2022,15,최종,외항,출항,2022-08-17 19:55:00,2022-08-18 23:55:00,MSN,02,신항 1부두 2선석,TIANJIN XINGANG PT,DALIAN
7481,부산,VRKR6,COSCO FOS,2022,15,최종,외항,입항,2022-08-17 19:55:00,2022-08-18 23:55:00,MSN,02,신항 1부두 2선석,TIANJIN XINGANG PT,DALIAN
9142,부산,VRKR6,COSCO FOS,2022,16,최종,외항,출항,2022-08-24 18:42:00,2022-08-26 03:00:00,MS6,02,신항 6부두 2선석,TIANJIN XINGANG PT,DALIAN
9143,부산,VRKR6,COSCO FOS,2022,16,최종,외항,입항,2022-08-24 18:42:00,2022-08-26 03:00:00,MS6,02,신항 6부두 2선석,TIANJIN XINGANG PT,DALIAN


In [180]:
d

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
3746,COSCO FOS,477947700,9484302.0,4253,KRBNP,9101100,119.6532,38.6811,13.3,96.0,2022-09-07 16:04:18.655000,2022-09-07 07:03:14,2022-09-07 07:01:11,under way using engine,913.689387,1281.471670
3747,COSCO FOS,477947700,9484302.0,4253,KRBNP,9101100,119.6971,38.6778,13.4,96.0,2022-09-07 16:13:37.571000,2022-09-07 07:12:40,2022-09-07 07:07:09,under way using engine,910.060522,1283.332353
3748,COSCO FOS,477947700,9484302.0,4253,KRBNP,9101100,119.7269,38.6754,13.3,96.0,2022-09-07 16:33:07.365000,2022-09-07 07:18:55,2022-09-07 07:25:08,under way using engine,907.591074,1272.918636
3749,COSCO FOS,477947700,9484302.0,4253,KRBNP,9101100,119.8695,38.6647,13.5,93.0,2022-09-07 16:55:45.254000,2022-09-07 07:49:15,2022-09-07 07:49:07,under way using engine,895.823260,1270.237747
3750,COSCO FOS,477947700,9484302.0,4253,KRBNP,9101100,120.2296,38.6566,13.4,90.0,2022-09-07 18:19:01.967000,2022-09-07 09:05:55,2022-09-07 08:49:04,under way using engine,867.116999,1222.775048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,COSCO FOS,477947700,9484302.0,4253,KRBNP,7210900,128.8295,35.0773,0.0,90.0,2023-01-14 19:43:12.000000,18:19.0,07:06.0,moored,12.010626,NaN
3,COSCO FOS,477947700,9484302.0,4253,KRBNP,7210900,128.8299,35.0774,0.0,91.0,2023-01-14 19:48:06.000000,06:13.0,19:05.0,moored,12.024540,NaN
4,COSCO FOS,477947700,9484302.0,4253,KRBNP,7210900,128.8297,35.0775,0.0,91.0,2023-01-14 19:52:54.000000,15:16.0,13:06.0,moored,12.034199,NaN
5,COSCO FOS,477947700,9484302.0,4253,KRBNP,7210900,128.8298,35.0772,0.0,91.0,2023-01-14 19:58:00.000000,12:18.0,13:07.0,moored,12.001653,NaN


In [161]:
path

'ZIM_CHARLESTON_voyage_df.csv'

In [136]:
port_mis_voyage_df.sort_values(by="df", ignore_index=)

,voyage_name,port_in_time,port_out_time,next_port,previous_port
0,9HA5144_2022_3,2022-12-18 21:10,2022-12-19 22:00,MANZANILLO,NINGBO


In [139]:
port_mis_df[port_mis_df.선명 == vessel_name]

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
36796,부산,9HA5144,AKADIMOS,2022,3,최종,외항,입항,2022-12-18 21:10,2022-12-19 22:00,MSN,01,신항 1부두 1선석,MANZANILLO,NINGBO
36797,부산,9HA5144,AKADIMOS,2022,3,최종,외항,출항,2022-12-18 21:10,2022-12-19 22:00,MSN,01,신항 1부두 1선석,MANZANILLO,NINGBO


In [137]:
voyage_df

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
0,AKADIMOS,215499000,9706308.0,9288,HKHKG,9140400,141.5711,13.4902,16.9,290.0,2022-09-11 00:10:21.029964,2022-09-10 13:39:31,2022-09-10 09:40:27,under way using engine,2709.886309,8042.979719
1,AKADIMOS,215499000,9706308.0,9288,HKHKG,9140400,133.1867,16.3133,18.0,NaN,2022-09-12 10:23:06.653045,2022-09-11 18:59:19,2022-09-10 09:40:27,under way using engine,2119.497891,6708.438624
2,AKADIMOS,215499000,9706308.0,9288,HKHKG,9150900,119.6501,19.7207,10.0,303.0,2022-09-14 21:35:01.165913,2022-09-14 11:20:12,2022-09-14 11:21:15,under way using engine,1919.934315,4650.832680
3,AKADIMOS,215499000,9706308.0,9288,HKHKG,9150900,118.7239,20.2812,11.7,303.0,2022-09-15 02:06:49.165779,2022-09-14 16:56:52,2022-09-14 04:57:16,under way using engine,1909.546525,4619.020125
4,AKADIMOS,215499000,9706308.0,9288,HKHKG,9150900,118.6643,20.3166,12.1,303.0,2022-09-15 02:36:52.010736,2022-09-14 17:16:54,2022-09-14 05:21:16,under way using engine,1909.143750,4647.113511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,AKADIMOS,215499000,9706308.0,9288,CN YTN # NO,12080500,114.1328,22.2357,7.6,128.0,2022-12-08 19:15:25.364118,2022-12-08 10:09:12.000000,2022-12-08 10:10:27.000000,under way using engine,2010.468078,225.093895
3821,AKADIMOS,215499000,9706308.0,9288,CN YTN # NO,12080500,114.1432,22.2280,6.9,127.0,2022-12-08 19:20:25.770692,2022-12-08 10:15:31.000000,2022-12-08 10:10:27.000000,under way using engine,2010.387941,238.830749
3822,AKADIMOS,215499000,8657668.0,9288,NAN,12100530,114.1293,22.2460,12.6,315.0,2022-12-10 14:52:03.533426,2022-12-10 05:46:39.000000,2022-12-10 05:51:29.000000,under way using engine,2009.858643,206.062257
3823,AKADIMOS,215499000,8657668.0,9288,NAN,12100530,114.1149,22.2676,12.9,332.0,2022-12-10 14:57:04.241803,2022-12-10 05:53:49.000000,2022-12-10 05:51:29.000000,under way using engine,2009.069339,207.625869


이 선박의 port-mis 데이터를 가져와서 몇 번 입항했는지 확인

In [162]:
COSCO_FOD_pmis_df = port_mis[port_mis["호출부호"] == 'VRKR6']
COSCO_FOD_pmis_df.reset_index(inplace=True, drop=True)

In [163]:
ZHU_pmis_df = port_mis[port_mis["호출부호"] == 'BTSJ']
ZHU_pmis_df.reset_index(inplace=True, drop=True)

In [164]:
COSCO_FOS_voyage_df["timestamp"] = COSCO_FOS_voyage_df[
    "timestamp"].apply(lambda x: pd.to_datetime(x) if len(x) == 26 else "")

NameError: name 'COSCO_FOS_voyage_df' is not defined

In [86]:
ZHU_CHENG_XIN_ZHOU_voyage_df["timestamp"] = ZHU_CHENG_XIN_ZHOU_voyage_df[
    "timestamp"].apply(lambda x: pd.to_datetime(x) if len(x) == 26 else "")

In [157]:
COSCO_FOS_voyage_df.drop(COSCO_FOS_voyage_df[
    COSCO_FOS_voyage_df["timestamp"] == ''].index,
                                  inplace=True)

In [88]:
ZHU_CHENG_XIN_ZHOU_voyage_df.drop(ZHU_CHENG_XIN_ZHOU_voyage_df[
    ZHU_CHENG_XIN_ZHOU_voyage_df["timestamp"] == ''].index,
                                  inplace=True)

In [160]:
COSCO_FOD_pmis_df2 = COSCO_FOD_pmis_df[["항명", "호출부호", "선명","입항횟수","입항횟수.1","구분",'외내','입출','입항일시','출항일시',"계선장소",'계선장소.1','계선장소.2','차항지','전출항지']]

voy_dict= {
          "voyage_name": [],
          "port_in_time" : [],
          "port_out_time" : [],
          "next_port" : [],
          "previous_port":[]}

for idx in COSCO_FOD_pmis_df2.index:
    if idx % 2 == 0:
        d = COSCO_FOD_pmis_df2.loc[idx:idx+1,:]
        display(d)
#         vessel_name = d["선명"].values[0]
        voyage_name = d["호출부호"].values[0] + "_" + str(d["입항횟수.1"].values[0])
        port_in_time = d['입항일시'].values[0]
        port_out_time= d['출항일시'].values[0]
        next_port= d['차항지'].values[0]
        previous_port= d['전출항지'].values[0]
        
        voy_dict["voyage_name"].append(voyage_name)
        voy_dict["port_in_time"].append(port_in_time)
        voy_dict["port_out_time"].append(port_out_time)
        voy_dict["next_port"].append(next_port)
        voy_dict["previous_port"].append(previous_port)
        

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
0,부산,VRKR6,COSCO FOS,2022,12,최종,외항,입항,2022-07-22 06:48,2022-07-23 17:56,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
1,부산,VRKR6,COSCO FOS,2022,12,최종,외항,출항,2022-07-22 06:48,2022-07-23 17:56,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
2,부산,VRKR6,COSCO FOS,2022,13,최종,외항,입항,2022-08-01 01:03,2022-08-02 17:56,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
3,부산,VRKR6,COSCO FOS,2022,13,최종,외항,출항,2022-08-01 01:03,2022-08-02 17:56,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
4,부산,VRKR6,COSCO FOS,2022,14,최종,외항,입항,2022-08-10 01:54,2022-08-11 14:54,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN
5,부산,VRKR6,COSCO FOS,2022,14,최종,외항,출항,2022-08-10 01:54,2022-08-11 14:54,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,DALIAN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
6,부산,VRKR6,COSCO FOS,2022,15,최종,외항,입항,2022-08-17 19:55,2022-08-18 23:55,MSN,02,신항 1부두 2선석,TIANJIN XINGANG PT,DALIAN
7,부산,VRKR6,COSCO FOS,2022,15,최종,외항,출항,2022-08-17 19:55,2022-08-18 23:55,MSN,02,신항 1부두 2선석,TIANJIN XINGANG PT,DALIAN


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
8,부산,VRKR6,COSCO FOS,2022,16,최종,외항,입항,2022-08-24 18:42,2022-08-26 03:00,MS6,02,신항 6부두 2선석,TIANJIN XINGANG PT,DALIAN
9,부산,VRKR6,COSCO FOS,2022,16,최종,외항,출항,2022-08-24 18:42,2022-08-26 03:00,MS6,02,신항 6부두 2선석,TIANJIN XINGANG PT,DALIAN


...

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
20,부산,VRKR6,COSCO FOS,2022,22,최종,외항,출항,2022-10-19 20:05,2022-10-23 11:32,MBR,E3,신감만부두 3선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT
21,부산,VRKR6,COSCO FOS,2022,22,최종,외항,입항,2022-10-19 20:05,2022-10-23 11:32,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
22,부산,VRKR6,COSCO FOS,2022,23,최종,외항,출항,2022-10-28 11:42,2022-10-30 16:40,MBR,E3,신감만부두 3선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT
23,부산,VRKR6,COSCO FOS,2022,23,최종,외항,입항,2022-10-28 11:42,2022-10-30 16:40,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
24,부산,VRKR6,COSCO FOS,2022,24,최종,외항,입항,2022-11-04 17:50,2022-11-08 09:54,MSN,02,신항 1부두 2선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT
25,부산,VRKR6,COSCO FOS,2022,24,최종,외항,출항,2022-11-04 17:50,2022-11-08 09:54,MBR,E3,신감만부두 3선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
26,부산,VRKR6,COSCO FOS,2022,25,최종,외항,출항,2022-11-14 16:20,2022-11-16 07:55,MBR,E3,신감만부두 3선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT
27,부산,VRKR6,COSCO FOS,2022,25,최종,외항,입항,2022-11-14 16:20,2022-11-16 07:55,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
28,부산,VRKR6,COSCO FOS,2022,26,최종,외항,출항,2022-11-21 17:51,2022-11-24 02:08,MBR,E3,신감만부두 3선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT
29,부산,VRKR6,COSCO FOS,2022,26,최종,외항,입항,2022-11-21 17:51,2022-11-24 02:08,MSN,01,신항 1부두 1선석,TIANJIN XINGANG PT,TIANJIN XINGANG PT


In [67]:
ZHU_pmis_df2 = ZHU_pmis_df[["항명", "호출부호", "선명","입항횟수","입항횟수.1","구분",'외내','입출','입항일시','출항일시',"계선장소",'계선장소.1','계선장소.2','차항지','전출항지']]

voy_dict= {
          "voyage_name": [],
          "port_in_time" : [],
          "port_out_time" : [],
          "next_port" : [],
          "previous_port":[]}

for idx in ZHU_pmis_df2.index:
    if idx % 2 == 0:
        d = ZHU_pmis_df2.loc[idx:idx+1,:]
        display(d)
#         vessel_name = d["선명"].values[0]
        voyage_name = d["호출부호"].values[0] + "_" + str(d["입항횟수.1"].values[0])
        port_in_time = d['입항일시'].values[0]
        port_out_time= d['출항일시'].values[0]
        next_port= d['차항지'].values[0]
        previous_port= d['전출항지'].values[0]
        
        voy_dict["voyage_name"].append(voyage_name)
        voy_dict["port_in_time"].append(port_in_time)
        voy_dict["port_out_time"].append(port_out_time)
        voy_dict["next_port"].append(next_port)
        voy_dict["previous_port"].append(previous_port)
        

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
0,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,22,최종,외항,출항,2022-07-22 23:40,2022-07-23 21:54,MSN,06,신항 2부두 3선석,OSAKA,HAKATA/FUKUOKA
1,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,22,최종,외항,입항,2022-07-22 23:40,2022-07-23 21:54,MSN,06,신항 2부두 3선석,OSAKA,HAKATA/FUKUOKA


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
2,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,23,최종,외항,출항,2022-07-29 11:55,2022-07-30 12:50,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA
3,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,23,최종,외항,입항,2022-07-29 11:55,2022-07-30 12:50,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
4,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,24,최종,외항,출항,2022-08-06 00:02,2022-08-06 22:00,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA
5,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,24,최종,외항,입항,2022-08-06 00:02,2022-08-06 22:00,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
6,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,25,최종,외항,입항,2022-08-12 07:55,2022-08-13 07:00,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA
7,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,25,최종,외항,출항,2022-08-12 07:55,2022-08-13 07:00,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
8,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,26,최종,외항,입항,2022-08-18 01:00,2022-08-18 23:00,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA
9,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,26,최종,외항,출항,2022-08-18 01:00,2022-08-18 23:00,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA


...

,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
24,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,34,최종,외항,입항,2022-10-20 23:31,2022-10-22 05:50,MSN,05,신항 2부두 2선석,OSAKA,HAKATA/FUKUOKA
25,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,34,최종,외항,출항,2022-10-20 23:31,2022-10-22 05:50,MSN,05,신항 2부두 2선석,OSAKA,HAKATA/FUKUOKA


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
26,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,35,최종,외항,출항,2022-10-27 22:45,2022-10-28 19:03,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA
27,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,35,최종,외항,입항,2022-10-27 22:45,2022-10-28 19:03,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
28,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,36,최종,외항,출항,2022-11-04 14:00,2022-11-05 17:55,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA
29,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,36,최종,외항,입항,2022-11-04 14:00,2022-11-05 17:55,MSN,04,신항 2부두 1선석,OSAKA,HAKATA/FUKUOKA


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
30,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,37,최종,외항,출항,2022-11-11 06:54,2022-11-12 09:50,MSN,05,신항 2부두 2선석,OSAKA,HAKATA/FUKUOKA
31,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,37,최종,외항,입항,2022-11-11 06:54,2022-11-12 09:50,MSN,05,신항 2부두 2선석,OSAKA,HAKATA/FUKUOKA


,항명,호출부호,선명,입항횟수,입항횟수.1,구분,외내,입출,입항일시,출항일시,계선장소,계선장소.1,계선장소.2,차항지,전출항지
32,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,38,최종,외항,입항,2022-11-17 23:56,2022-11-19 13:00,MSN,05,신항 2부두 2선석,OSAKA,HAKATA/FUKUOKA
33,부산,BTSJ,ZHU CHENG XIN ZHOU,2022,38,최종,외항,출항,2022-11-17 23:56,2022-11-19 13:00,MSN,02,신항 1부두 2선석,OSAKA,HAKATA/FUKUOKA


In [161]:
voy_df_test = pd.DataFrame(voy_dict)
voy_df_test

,voyage_name,port_in_time,port_out_time,next_port,previous_port
0,VRKR6_12,2022-07-22 06:48,2022-07-23 17:56,TIANJIN XINGANG PT,DALIAN
1,VRKR6_13,2022-08-01 01:03,2022-08-02 17:56,TIANJIN XINGANG PT,DALIAN
2,VRKR6_14,2022-08-10 01:54,2022-08-11 14:54,TIANJIN XINGANG PT,DALIAN
3,VRKR6_15,2022-08-17 19:55,2022-08-18 23:55,TIANJIN XINGANG PT,DALIAN
4,VRKR6_16,2022-08-24 18:42,2022-08-26 03:00,TIANJIN XINGANG PT,DALIAN
5,VRKR6_17,2022-08-30 19:35,2022-09-04 11:48,TIANJIN XINGANG PT,TIANJIN XINGANG PT
6,VRKR6_18,2022-09-11 08:23,2022-09-15 03:52,TIANJIN XINGANG PT,TIANJIN XINGANG PT
7,VRKR6_19,2022-09-24 14:15,2022-09-27 16:40,TIANJIN XINGANG PT,TIANJIN XINGANG PT
8,VRKR6_20,2022-10-04 21:00,2022-10-06 19:50,TIANJIN XINGANG PT,TIANJIN XINGANG PT
9,VRKR6_21,2022-10-12 00:50,2022-10-13 20:42,TIANJIN XINGANG PT,TIANJIN XINGANG PT


In [69]:
voy_df_test = pd.DataFrame(voy_dict)
voy_df_test

,voyage_name,port_in_time,port_out_time,next_port,previous_port
0,BTSJ_22,2022-07-22 23:40,2022-07-23 21:54,OSAKA,HAKATA/FUKUOKA
1,BTSJ_23,2022-07-29 11:55,2022-07-30 12:50,OSAKA,HAKATA/FUKUOKA
2,BTSJ_24,2022-08-06 00:02,2022-08-06 22:00,OSAKA,HAKATA/FUKUOKA
3,BTSJ_25,2022-08-12 07:55,2022-08-13 07:00,OSAKA,HAKATA/FUKUOKA
4,BTSJ_26,2022-08-18 01:00,2022-08-18 23:00,OSAKA,HAKATA/FUKUOKA
5,BTSJ_27,2022-08-26 08:30,2022-08-27 00:03,OSAKA,HAKATA/FUKUOKA
6,BTSJ_28,2022-09-01 05:11,2022-09-02 00:53,OSAKA,HAKATA/FUKUOKA
7,BTSJ_29,2022-09-11 07:05,2022-09-12 03:05,OSAKA,HAKATA/FUKUOKA
8,BTSJ_30,2022-09-16 22:53,2022-09-17 22:52,OSAKA,HAKATA/FUKUOKA
9,BTSJ_31,2022-09-26 22:40,2022-09-28 04:50,OSAKA,HAKATA/FUKUOKA


In [163]:
for idx in voy_df_test.index:
    if idx == 0:
        port_in_time = pd.to_datetime(
            voy_df_test.loc[idx, "port_in_time"]) + datetime.timedelta(days=1)
        df = COSCO_FOS_voyage_df[
            COSCO_FOS_voyage_df.eta < str(port_in_time)]
        print(port_in_time)
        
        display(df)
    else:
        port_in_time = pd.to_datetime(
            voy_df_test.loc[idx, "port_in_time"])
        prev_out_time = pd.to_datetime(voy_df_test.loc[idx - 1,
                                                       "port_out_time"])
#         print(port_in_time)
        voyage_name = voy_df_test.loc[idx,"voyage_name"]
        df = COSCO_FOS_voyage_df[
            (COSCO_FOS_voyage_df["timestamp"]<
             datetime.datetime.fromtimestamp(port_in_time.timestamp()))
            & (COSCO_FOS_voyage_df["timestamp"] >
               datetime.datetime.fromtimestamp(prev_out_time.timestamp()))]
        if len(df) != 0:
            
            print("전항차 출항시각", prev_out_time)
            print("입항시각" , port_in_time)
            
            display(df.sort_values(by="timestamp"))
            df.to_csv(f"COSCO_FOS_" + voyage_name + ".csv", encoding="utf-8 sig", header=True, index=False)
            print("="*100)

2022-07-23 06:48:00


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption


전항차 출항시각 2022-09-04 11:48:00
입항시각 2022-09-11 08:23:00


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
3761,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 11:00:00,119.6532,38.6811,13.3,96.0,2022-09-07 16:04:18.655000,2022-09-07 07:03:14,2022-09-07 07:01:11,under way using engine,913.689387,1281.471670
3762,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 11:00:00,119.6971,38.6778,13.4,96.0,2022-09-07 16:13:37.571000,2022-09-07 07:12:40,2022-09-07 07:07:09,under way using engine,910.060522,1283.332353
3763,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 11:00:00,119.7269,38.6754,13.3,96.0,2022-09-07 16:33:07.365000,2022-09-07 07:18:55,2022-09-07 07:25:08,under way using engine,907.591074,1272.918636
3764,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 11:00:00,119.8695,38.6647,13.5,93.0,2022-09-07 16:55:45.254000,2022-09-07 07:49:15,2022-09-07 07:49:07,under way using engine,895.823260,1270.237747
3765,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 11:00:00,120.2296,38.6566,13.4,90.0,2022-09-07 18:19:01.967000,2022-09-07 09:05:55,2022-09-07 08:49:04,under way using engine,867.116999,1222.775048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 10:00:00,127.4207,34.1230,11.8,69.0,2022-09-10 22:09:45.198728,2022-09-10 13:09:19,2022-09-09 02:43:07,under way using engine,158.973966,207.932024
136,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 10:00:00,127.5280,34.1620,11.6,71.0,2022-09-10 22:39:49.271635,2022-09-10 13:39:00,2022-09-10 01:19:06,under way using engine,148.483546,192.789162
137,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 10:00:00,127.6465,34.1893,12.0,81.0,2022-09-10 23:09:55.406167,2022-09-10 14:09:28,2022-09-10 01:19:06,under way using engine,138.055303,181.996885
138,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-09-10 10:00:00,127.7650,34.2076,11.9,79.0,2022-09-10 23:39:56.666533,2022-09-10 14:39:27,2022-09-10 02:31:07,under way using engine,128.437050,168.642149


전항차 출항시각 2022-09-15 03:52:00
입항시각 2022-09-24 14:15:00


...

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
1699,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-01 01:00:00,126.4761,34.0970,20.2,282.0,2022-10-31 01:44:32.801828,2022-10-30 16:41:25,2022-10-30 01:55:08,under way using engine,235.557924,529.149365
1700,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-01 01:00:00,126.4315,34.1047,20.2,282.0,2022-10-31 01:49:35.660712,2022-10-30 16:48:08,2022-10-30 01:55:08,under way using engine,238.931185,536.726944
1701,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-01 01:00:00,126.3895,34.1077,20.2,271.0,2022-10-31 01:54:38.504486,2022-10-30 16:54:25,2022-10-30 01:55:08,under way using engine,242.321724,544.343338
1702,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-01 01:00:00,126.3598,34.1082,20.1,271.0,2022-10-31 01:59:42.326869,2022-10-30 16:58:50,2022-10-30 01:55:08,under way using engine,244.800071,545.915560
1703,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-01 01:00:00,126.3213,34.1087,20.2,271.0,2022-10-31 02:04:44.283430,2022-10-30 17:04:31,2022-10-30 01:55:08,under way using engine,248.026798,557.159024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2172,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-04 08:00:00,128.8247,35.0773,0.0,91.0,2022-11-04 20:13:03.886599,2022-11-04 11:09:51,2022-11-04 11:07:05,under way using engine,11.985248,NaN
2173,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-04 08:00:00,128.8246,35.0772,0.0,91.0,2022-11-04 20:18:07.878970,2022-11-04 11:15:24,2022-11-04 11:07:05,under way using engine,11.973778,NaN
2174,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-04 08:00:00,128.8249,35.0774,0.1,91.0,2022-11-04 20:58:30.521894,2022-11-04 11:57:20,2022-11-04 11:49:08,under way using engine,11.997099,957.274033
2175,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-04 08:00:00,128.8249,35.0775,0.0,91.0,2022-11-04 21:03:31.914746,2022-11-04 12:00:35,2022-11-04 12:01:08,under way using engine,12.008209,NaN


전항차 출항시각 2022-11-08 09:54:00
입항시각 2022-11-14 16:20:00


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
2355,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.5569,34.0799,18.4,280.0,2022-11-08 18:56:13.959541,2022-11-08 09:55:53,2022-11-08 07:31:08,under way using engine,229.666166,452.388243
2356,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.5261,34.0849,18.3,285.0,2022-11-08 19:01:17.137621,2022-11-08 10:01:01,2022-11-08 07:31:08,under way using engine,231.973583,453.620310
2357,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.4980,34.0905,18.5,280.0,2022-11-08 19:06:19.944782,2022-11-08 10:05:41,2022-11-08 07:31:08,under way using engine,234.039994,464.373403
2358,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.4644,34.0925,18.8,275.0,2022-11-08 19:11:23.235915,2022-11-08 10:11:06,2022-11-08 07:31:08,under way using engine,236.745812,480.132626
2359,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-09 09:00:00,126.4325,34.0943,18.9,275.0,2022-11-08 19:16:26.088005,2022-11-08 10:16:10,2022-11-08 07:31:08,under way using engine,239.327692,488.925120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-14 05:50:00,128.8127,35.0728,6.0,80.0,2022-11-14 15:45:30.478230,2022-11-14 06:42:40,2022-11-14 06:43:07,under way using engine,11.491783,17.159610
2839,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-14 05:50:00,128.8261,35.0755,1.4,76.0,2022-11-14 16:00:40.036669,2022-11-14 06:58:29,2022-11-14 06:49:08,under way using engine,11.791119,67.307567
2840,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-14 05:50:00,128.8294,35.0774,0.0,90.0,2022-11-14 16:41:02.521600,2022-11-14 07:39:30,2022-11-14 07:37:07,moored,12.021023,NaN
2841,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-14 05:50:00,128.8293,35.0774,0.1,91.0,2022-11-14 17:26:27.030687,2022-11-14 08:24:24,2022-11-14 08:25:09,moored,12.020340,959.128537


전항차 출항시각 2022-11-16 07:55:00
입항시각 2022-11-21 17:51:00


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
2961,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,127.0882,34.1223,16.0,270.0,2022-11-16 16:56:08.041108,2022-11-16 07:55:49,2022-11-16 04:19:07,under way using engine,184.436450,306.205403
2962,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,127.0609,34.1211,16.1,267.0,2022-11-16 17:01:11.033821,2022-11-16 08:00:55,2022-11-16 04:19:07,under way using engine,186.658612,312.035073
2963,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,126.6711,34.0831,16.1,262.0,2022-11-16 18:16:53.451054,2022-11-16 09:14:25,2022-11-16 04:19:07,under way using engine,220.108466,367.952814
2964,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,126.6067,34.0795,16.0,277.0,2022-11-16 18:26:58.801742,2022-11-16 09:26:25,2022-11-16 04:19:07,under way using engine,225.577733,374.509054
2965,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,126.5502,34.0820,16.2,280.0,2022-11-16 18:42:07.917210,2022-11-16 09:37:09,2022-11-16 04:19:07,under way using engine,230.117608,387.353392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3489,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8286,35.0767,0.9,80.0,2022-11-21 18:05:52.289081,2022-11-21 08:45:01,2022-11-21 08:37:08,under way using engine,11.938104,105.884565
3490,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8303,35.0770,0.1,91.0,2022-11-21 20:12:03.039219,2022-11-21 10:46:03,2022-11-21 10:49:07,moored,11.983151,956.161110
3491,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8302,35.0772,0.0,91.0,2022-11-21 20:17:05.890117,2022-11-21 10:52:05,2022-11-21 10:55:06,moored,12.004568,NaN
3492,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8300,35.0772,0.0,91.0,2022-11-21 20:42:20.130436,2022-11-21 11:15:59,2022-11-21 11:13:06,moored,12.003097,NaN


In [147]:
for idx in voy_df_test.index:
    if idx == 0:
        port_in_time = pd.to_datetime(
            voy_df_test.loc[idx, "port_in_time"]) + datetime.timedelta(days=1)
        df = ZHU_CHENG_XIN_ZHOU_voyage_df[
            ZHU_CHENG_XIN_ZHOU_voyage_df.eta < str(port_in_time)]
        print(port_in_time)
        
        display(df)
    else:
        port_in_time = pd.to_datetime(
            voy_df_test.loc[idx, "port_in_time"])
        prev_out_time = pd.to_datetime(voy_df_test.loc[idx - 1,
                                                       "port_out_time"])
#         print(port_in_time)
        voyage_name = voy_df_test.loc[idx,"voyage_name"]
        df = ZHU_CHENG_XIN_ZHOU_voyage_df[
            (ZHU_CHENG_XIN_ZHOU_voyage_df["timestamp"]<
             datetime.datetime.fromtimestamp(port_in_time.timestamp()))
            & (ZHU_CHENG_XIN_ZHOU_voyage_df["timestamp"] >
               datetime.datetime.fromtimestamp(prev_out_time.timestamp()))]
        if len(df) != 0:
            
            print("전항차 출항시각", prev_out_time)
            print("입항시각" , port_in_time)
            
            display(df.sort_values(by="timestamp"))
            df.to_csv(f"ZHU_CHENG_XIN_ZHOU_" + voyage_name + ".csv", encoding="utf-8 sig", header=True, index=False)
            print("="*100)

2022-07-23 23:40:00


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
122,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-07-22 09:00:00,129.176,34.7683,3.6,311.0,2022-07-21 14:53:26.001000,2022-07-21 05:45:43,2022-07-21 05:30:26,under way using engine,39.485762,66.301892


전항차 출항시각 2022-09-02 00:53:00
입항시각 2022-09-11 07:05:00


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
3896,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPHKT,2022-09-08 01:00:00,131.9043,33.6274,0.5,33.0,2022-09-07 16:04:18.655000,2022-09-07 07:00:59,2022-09-07 06:50:57,under way using engine,320.249767,4863.371058
3897,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPHKT,2022-09-08 01:00:00,131.9063,33.6272,0.4,31.0,2022-09-07 16:13:37.571000,2022-09-07 07:13:03,2022-09-07 07:09:07,under way using engine,320.422954,6082.285479
3898,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPHKT,2022-09-08 01:00:00,131.9090,33.6270,0.4,29.0,2022-09-07 16:33:07.365000,2022-09-07 07:28:47,2022-09-07 07:27:18,under way using engine,320.653051,6086.653202
3899,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPHKT,2022-09-08 01:00:00,131.9118,33.6269,0.4,28.0,2022-09-07 16:55:45.254000,2022-09-07 07:44:26,2022-09-07 07:45:26,under way using engine,320.885986,6091.074782
3900,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPHKT,2022-09-08 01:00:00,131.7962,33.7064,11.9,308.0,2022-09-07 18:19:01.967000,2022-09-07 09:16:34,2022-09-07 09:16:21,under way using engine,307.274795,388.312543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-09-11 07:00:00,128.8571,34.6294,1.5,324.0,2022-09-10 22:09:45.198728,2022-09-10 13:06:45,2022-09-10 12:46:55,under way using engine,37.984451,192.643228
217,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-09-11 07:00:00,128.8442,34.6283,1.0,326.0,2022-09-10 22:39:49.271635,2022-09-10 13:37:44,2022-09-10 01:29:39,under way using engine,38.018107,288.821919
218,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-09-11 07:00:00,128.8413,34.6367,9.2,74.0,2022-09-10 23:09:55.406167,2022-09-10 14:07:55,2022-09-10 01:53:42,under way using engine,37.070562,44.457122
219,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-09-11 07:00:00,128.9532,34.6528,10.4,82.0,2022-09-10 23:39:56.666533,2022-09-10 14:38:25,2022-09-10 02:29:58,under way using engine,37.282905,45.036088


전항차 출항시각 2022-09-12 03:05:00
입항시각 2022-09-16 22:53:00


...

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
1855,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-10-30 05:00:00,130.7386,34.0765,8.8,112.0,2022-10-29 04:03:50.809631,2022-10-28 19:02:15,2022-10-28 06:53:59,under way using engine,201.921631,243.306716
1856,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-10-30 05:00:00,130.7658,34.0653,8.8,112.0,2022-10-29 04:13:56.421557,2022-10-28 19:12:26,2022-10-28 07:11:58,under way using engine,204.714057,246.671467
1857,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-10-30 05:00:00,130.7790,34.0597,8.9,113.0,2022-10-29 04:19:00.252878,2022-10-28 19:17:24,2022-10-28 07:11:58,under way using engine,206.078488,247.929453
1858,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-10-30 05:00:00,130.7885,34.0557,9.0,113.0,2022-10-29 04:24:02.391251,2022-10-28 19:20:59,2022-10-28 07:11:58,under way using engine,207.058835,248.783864
1859,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-10-30 05:00:00,130.8207,34.0422,9.3,113.0,2022-10-29 04:34:07.913568,2022-10-28 19:32:34,2022-10-28 07:29:17,under way using engine,210.378808,252.150997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-04 01:00:00,128.8183,35.0773,0.4,271.0,2022-11-04 13:59:33.183462,2022-11-04 04:58:30,2022-11-04 04:56:41,under way using engine,11.976168,227.332255
2505,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-04 01:00:00,128.8181,35.0775,0.0,269.0,2022-11-04 16:25:56.063538,2022-11-04 07:24:49,2022-11-04 07:24:58,moored,11.998579,NaN
2506,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-04 01:00:00,128.8181,35.0774,0.0,269.0,2022-11-04 18:47:16.806349,2022-11-04 09:42:10,2022-11-04 09:44:20,moored,11.987460,NaN
2507,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-04 01:00:00,128.8182,35.0774,0.0,269.0,2022-11-04 18:52:18.020141,2022-11-04 09:51:13,2022-11-04 09:50:22,moored,11.987370,NaN


전항차 출항시각 2022-11-05 17:55:00
입항시각 2022-11-11 06:54:00


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
2509,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-07 05:00:00,131.2245,33.8430,16.0,112.0,2022-11-06 08:47:49.515772,2022-11-05 23:37:30,2022-11-05 11:44:01,under way using engine,253.713539,407.379358
2510,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-07 05:00:00,131.5847,33.7828,15.4,101.0,2022-11-06 10:44:43.157567,2022-11-06 01:39:40,2022-11-06 01:39:02,under way using engine,286.024124,440.737609
2511,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-07 05:00:00,131.6300,33.7738,15.6,107.0,2022-11-06 10:49:45.613821,2022-11-06 01:48:35,2022-11-06 01:39:02,under way using engine,290.187791,453.268816
2512,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-07 05:00:00,131.7073,33.7502,15.8,122.0,2022-11-06 11:04:54.155885,2022-11-06 02:04:26,2022-11-06 02:03:27,under way using engine,297.729535,471.476089
2513,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-07 05:00:00,131.7099,33.7487,15.8,120.0,2022-11-06 11:09:57.050988,2022-11-06 02:04:58,2022-11-06 02:09:22,under way using engine,298.020028,471.936107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-11 06:00:00,128.8001,35.0707,3.8,61.0,2022-11-11 06:41:01.091976,2022-11-10 21:36:40,2022-11-10 09:31:15,under way using engine,11.379081,23.461837
2955,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-11 06:00:00,128.8075,35.0743,3.1,62.0,2022-11-11 06:46:03.604314,2022-11-10 21:43:21,2022-11-10 09:31:15,under way using engine,11.692870,29.134746
2956,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-11 06:00:00,128.8118,35.0774,0.1,92.0,2022-11-11 06:56:09.371501,2022-11-10 21:52:59,2022-11-10 09:55:15,under way using engine,12.007054,911.640709
2957,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-11 06:00:00,128.8119,35.0774,0.0,89.0,2022-11-11 07:41:34.983984,2022-11-10 22:37:49,2022-11-10 10:36:22,moored,12.006529,NaN


전항차 출항시각 2022-11-12 09:50:00
입항시각 2022-11-17 23:56:00


,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
3021,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-13 05:00:00,130.7776,34.0635,8.2,118.0,2022-11-12 18:50:17.003869,2022-11-12 09:45:46,2022-11-12 09:48:59,under way using engine,205.755989,251.643372
3022,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-13 05:00:00,130.7995,34.0529,8.2,124.0,2022-11-12 18:55:19.038724,2022-11-12 09:54:45,2022-11-12 09:51:27,under way using engine,208.092468,254.500930
3023,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-13 05:00:00,130.8054,34.0494,8.2,118.0,2022-11-12 19:00:22.794884,2022-11-12 09:57:39,2022-11-12 09:51:27,under way using engine,208.757682,255.314499
3024,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-13 05:00:00,130.8142,34.0452,8.2,121.0,2022-11-12 19:05:25.123531,2022-11-12 10:01:17,2022-11-12 09:51:27,under way using engine,209.693344,256.458830
3025,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,JPOSA,2022-11-13 05:00:00,130.8342,34.0323,9.0,137.0,2022-11-12 19:10:28.167013,2022-11-12 10:09:44,2022-11-12 10:06:56,under way using engine,212.006309,254.728318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-17 10:00:00,128.7836,35.0513,8.8,8.0,2022-11-17 23:08:07.738297,2022-11-17 14:05:24,2022-11-17 02:05:46,under way using engine,9.653543,11.632097
3448,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-17 10:00:00,128.7905,35.0689,5.4,58.0,2022-11-17 23:18:13.383658,2022-11-17 14:13:43,2022-11-17 02:17:55,under way using engine,11.352032,17.423879
3449,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-17 10:00:00,128.8030,35.0730,3.1,69.0,2022-11-17 23:23:16.128957,2022-11-17 14:22:45,2022-11-17 02:17:55,under way using engine,11.594743,28.890245
3450,ZHU CHENG XIN ZHOU,413556780,9252993.0,2524,KRBNP,2022-11-17 10:00:00,128.8074,35.0747,1.8,68.0,2022-11-17 23:33:21.939623,2022-11-17 14:28:44,2022-11-17 02:24:04,under way using engine,11.738003,49.679841


In [144]:
df.reset_index(inplace=True, drop=True)
df.to_csv("test_ZHU.csv", encoding="utf-8 sig", index=False, header=True)